This is an example notebook on how to use `SearchSpace`

In [ ]:
# import necessary libraries
from HPSearchSpace import SearchSpace
import hyperopt
import optuna

from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score

To demonstrate how to use `SearchSpace` for different hyperparameter optimization libraries, we use the `iris` dataset to perform a simple classification task.

In [1]:
iris = load_iris()
x, y = iris.data, iris.target

The first step is to define what machine learning methods and what hyperparameter to optimize. The configuration is already stored in `example.yaml` file. We can simply load it.

In [ ]:
search_space = SearchSpace(config_file='example.yaml')

Then, we can use `get_hyperopt_space` and `get_optuna_space` to get the search space for `hyperopt` and `optuna` respectively.

In [ ]:
hyperopt_space = search_space.get_hyperopt_space()
optuna_space = search_space.get_optuna_space()

Now, we can use the search space to optimize the hyperparameters of a machine learning model.

- Hyperopt

In [ ]:
# hyperopt objective function
def objective(config):  
    # config is the sampled hyperparameters from the search space.
    # we want to maximize the accuracy of the model, however, hyperopt minimizes the objective function, so we use negative accuracy
    model = config["model"]
    params = config["params"]
    model = model(**params)
    score = cross_val_score(model, x, y, cv=5)
    return -score.mean()

In [ ]:
# hyperopt optimization
trial_hp = hyperopt.Trials()
best_hyperopt = hyperopt.fmin(objective, hyperopt_space, algo=hyperopt.tpe.suggest, max_evals=10, trials=trial_hp)

- Optuna

In [ ]:
# optuna objective function
def objective_op(trial):
    # similar to hyperopt, but the objective function takes in a trial object, and sampling in done using the trial object.
    # A workaround is to use the optuna_space as a function where it contains the search space, and it takes in trial object and returns the sampled hyperparameters.
    config = optuna_space(trial)
    return objective(config)

In [ ]:
# optuna optimization
study = optuna.create_study(direction="minimize")
study.optimize(objective_op, n_trials=10)